<a href="https://colab.research.google.com/github/NewburghCat/dalmatians_sales/blob/main/DataSet_Prj_2A_DATACLEANED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

JodyAnn Bradford

Project 2: E-Commerce DataSet 

August 5th 2022.


In [25]:
# Project_2 : Data Set - E-Commerce Data 

This Data was chosen to facilitate an anaylsis which predicts a revelant product type according to the region and shipping. 

Description: 

An international e-commerce company based customer insight data. This company sells electronic products. 

Regression 

The target vector is Reached.on.Time_Y.N as either Yes (1) or No (2).  

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector,make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_squared_error

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# load data 

file = '/content/drive/MyDrive/Colab Notebooks/Data Science - Machine Learning/WK 3/PROJECT2/Train_E-CommerceShippingData.csv'
df = pd.read_csv(file)
df.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


Data Cleaning 

In [8]:
# Checking for duplicates 

df.duplicated().sum()

0

No duplicates found 

In [10]:
# Checking data types 

df.isna().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
Reached.on.Time_Y.N    0
dtype: int64

No Missing values 

The Unnecessary columns are Customer care calls, customer rating, prior purchases, discount offered and Gender. The target column is trying to figure out the shipment arrival time. These columns will only act as overfitted data. 

In [11]:
# Deleting unnecessary columns

df = df.drop(columns=['Customer_care_calls','Customer_rating', 'Prior_purchases', 'Discount_offered', 'Gender'])
df

,ID,Warehouse_block,Mode_of_Shipment,Cost_of_the_Product,Product_importance,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,177,low,1233,1
1,2,F,Flight,216,low,3088,1
2,3,A,Flight,183,low,3374,1
3,4,B,Flight,176,medium,1177,1
4,5,C,Flight,184,medium,2484,1
...,...,...,...,...,...,...,...
10994,10995,A,Ship,252,medium,1538,1
10995,10996,B,Ship,232,medium,1247,0
10996,10997,C,Ship,242,low,1155,0
10997,10998,F,Ship,223,medium,1210,0


In [12]:
# create a dtype filter to be used to count their values 

dtypes = df.dtypes
header_columns = dtypes[dtypes=='object'].index
header_columns

for all in header_columns:
  print(f'- column= {all}')
  print(df[all].value_counts(dropna=False))
  print('\n\n')

- column= Warehouse_block
F    3666
D    1834
A    1833
B    1833
C    1833
Name: Warehouse_block, dtype: int64



- column= Mode_of_Shipment
Ship      7462
Flight    1777
Road      1760
Name: Mode_of_Shipment, dtype: int64



- column= Product_importance
low       5297
medium    4754
high       948
Name: Product_importance, dtype: int64





In [13]:
# Checking dataset 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   10999 non-null  int64 
 1   Warehouse_block      10999 non-null  object
 2   Mode_of_Shipment     10999 non-null  object
 3   Cost_of_the_Product  10999 non-null  int64 
 4   Product_importance   10999 non-null  object
 5   Weight_in_gms        10999 non-null  int64 
 6   Reached.on.Time_Y.N  10999 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 601.6+ KB


In [14]:
df.describe()

,ID,Cost_of_the_Product,Weight_in_gms,Reached.on.Time_Y.N
count,10999.00000,10999.000000,10999.000000,10999.000000
mean,5500.00000,210.196836,3634.016729,0.596691
std,3175.28214,48.063272,1635.377251,0.490584
min,1.00000,96.000000,1001.000000,0.000000
25%,2750.50000,169.000000,1839.500000,0.000000
50%,5500.00000,214.000000,4149.000000,1.000000
75%,8249.50000,251.000000,5050.000000,1.000000
max,10999.00000,310.000000,7846.000000,1.000000


In [15]:
## Create X and y 

target = 'Reached.on.Time_Y.N'
X = df.drop(columns=target)
y = df[target]

In [16]:
## train-test-split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.shape

(8249, 6)

Feature Data Types

Numeric Features: Warehouse_block, Mode_of_Shipment, Product_Importance, Discount_offered

Nominal/Categorical Features: 8 Columns 

Preprocessor

A. OneHotEncoder 
B. Scaler to scale different scales. 

* No Missing values 

In [17]:

## Make a column selector for categories & test it 

cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [18]:
## Instantiate OneHotEncoder and StandardScaler

ohe_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
scaler = StandardScaler()

## Create tuples of (transformer, selector) - ColumnTransformer

cat_tuple = (ohe_encoder, cat_selector)
num_tuple = (scaler, num_selector)

In [19]:
## ColumnTransformer

preprocessor = make_column_transformer(cat_tuple, num_tuple)

Modeling

In [20]:
## Make and fit a linear regression model
reg = LinearRegression()
reg_pipe = make_pipeline(preprocessor, reg)

reg_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ff68a2c9850>),
                                                 ('standardscaler',
                                                  StandardScaler(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ff68a2c9150>)])),
                ('linearregression', LinearRegression())])

Evaluating Model performance - using functions from metrics 

In [21]:
## Model Performance - using builtin .score()

print(f"Train R-Squared: { round( reg_pipe.score(X_train, y_train),3) }")
print(f"Test R-Squared: { round( reg_pipe.score(X_test, y_test),3) }")

Train R-Squared: 0.197
Test R-Squared: 0.195


In [22]:
# get predictions for train and test data

y_h_train = reg_pipe.predict(X_train)
y_h_test = reg_pipe.predict(X_test)

In [23]:
## Get r-square for train vs test
print(f"Train R-Squared: { round( r2_score(y_train, y_h_train), 3)}")
print(f"Test R-Squared: { round( r2_score(y_test, y_h_test), 3)}")

Train R-Squared: 0.197
Test R-Squared: 0.195


In [24]:
## Get RMSE for train vs test

rmse_train =  mean_squared_error(y_train, y_h_train, squared=False)
rmse_test = mean_squared_error(y_test, y_h_test, squared=False)
print(f"Train RMSE: { round(rmse_train, 2)}")
print(f"Test RMSE: { round( rmse_test, 2)}")

Train RMSE: 0.44
Test RMSE: 0.44
